In [1]:
MAX_SEQUENCE_LENGTH = 200 # 每条新闻最大长度
EMBEDDING_DIM = 300 # 词向量空间维度
VALIDATION_SPLIT = 0.2 # 验证集占训练集比例
#TEST_SPLIT = 0.3 # 测试集占全数据集比例

In [2]:
#import os
import pandas as pd
import numpy as np
#import inspect
from tqdm import trange
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from sklearn.preprocessing import LabelEncoder as LE
from keras.models import load_model
from sklearn.model_selection import train_test_split as tts
from time import time

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
output=pd.read_csv('./output.csv')#已经在BOW_SVM中预处理好的文本数据

X=output['doc'].tolist()
y=output['label'].tolist()

le = LE()
y=le.fit_transform(y)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

x_train, x_test, y_train, y_test = tts(data, labels, test_size=0.3, random_state=42)
print ('train docs: '+str(len(x_train)))
print ('test docs: '+str(len(x_test)))

Found 106298 unique tokens.
Shape of data tensor: (18847, 200)
Shape of label tensor: (18847, 20)
train docs: 13192
test docs: 5655


In [22]:
len(word_index)

106298

# 载入预训练词向量

In [4]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, BatchNormalization, Activation, Concatenate, Add, GlobalMaxPooling1D
from keras.models import Sequential, Model
from keras.utils import plot_model
import gensim

#w2v_model = gensim.models.Word2Vec.load('./en_1000_no_stem/en.model')
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)

E:\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 开始训练

In [7]:
start=time()
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items(): 
    if word in w2v_model:
        embedding_matrix[i] = np.asarray(w2v_model[word],
                                         dtype='float32')
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)#trainable的意义是什么？

model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1))
#model.add(GlobalMaxPooling1D())
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()
#plot_model(model, to_file='model.png',show_shapes=True)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model.fit(x_train, y_train, validation_split=VALIDATION_SPLIT, epochs=5, batch_size=128)
loss,accuracy = model.evaluate(x_test,y_test, verbose=0)
end=time()
print('Test score:', loss)
print('Test accuracy:', accuracy)
print("training and test time:{:.2f}s".format(end-start))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 300)          31889700  
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 300)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 198, 250)          225250    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 66, 250)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16500)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               4950300   
_________________________________________________________________
dense_2 (Dense)              (None, 20)                6020      
Total para

10553/10553 [==============================] - ETA: 3:23 - loss: 0.0191 - acc: 1.000 - ETA: 3:11 - loss: 0.0149 - acc: 1.000 - ETA: 3:07 - loss: 0.0153 - acc: 1.000 - ETA: 3:04 - loss: 0.0165 - acc: 1.000 - ETA: 3:00 - loss: 0.0185 - acc: 1.000 - ETA: 2:57 - loss: 0.0216 - acc: 1.000 - ETA: 2:55 - loss: 0.0229 - acc: 1.000 - ETA: 2:52 - loss: 0.0232 - acc: 1.000 - ETA: 2:50 - loss: 0.0218 - acc: 1.000 - ETA: 2:47 - loss: 0.0219 - acc: 1.000 - ETA: 2:45 - loss: 0.0229 - acc: 1.000 - ETA: 2:42 - loss: 0.0224 - acc: 1.000 - ETA: 2:40 - loss: 0.0230 - acc: 1.000 - ETA: 2:38 - loss: 0.0228 - acc: 1.000 - ETA: 2:35 - loss: 0.0227 - acc: 1.000 - ETA: 2:33 - loss: 0.0239 - acc: 0.999 - ETA: 2:30 - loss: 0.0241 - acc: 0.999 - ETA: 2:28 - loss: 0.0238 - acc: 0.999 - ETA: 2:27 - loss: 0.0236 - acc: 0.999 - ETA: 2:25 - loss: 0.0231 - acc: 0.999 - ETA: 2:23 - loss: 0.0230 - acc: 0.999 - ETA: 2:20 - loss: 0.0237 - acc: 0.999 - ETA: 2:18 - loss: 0.0236 - acc: 0.999 - ETA: 2:15 - loss: 0.0234 - acc: 0

# 使用merge方法

In [8]:
from keras.layers import GlobalMaxPooling1D,GlobalMaxPooling2D

In [9]:
start=time()
convs = []
 
filter_sizes = [2,3,4,5]

input_img = Input(shape=(MAX_SEQUENCE_LENGTH,))
embedded_sequences = embedding_layer(input_img)
for x in filter_sizes:
    
    l_conv = Conv1D(nb_filter=250,filter_length=x,activation='relu')(embedded_sequences)
    #l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
 
    l_pool = MaxPooling1D(5)(l_conv)
 
    convs.append(l_pool)
    
l_merge = Concatenate()(convs) 

l_flat = Flatten()(l_merge)

l_dense = Dense(EMBEDDING_DIM, activation='relu')(l_flat)
 
preds = Dense(labels.shape[1], activation='softmax')(l_dense)

merge_model = Model(input_img, preds)

merge_model.summary() 

merge_model.compile(loss='categorical_crossentropy',
 
              optimizer='adam',
 
              metrics=['acc'])

merge_model.fit(x_train, y_train, validation_split=VALIDATION_SPLIT, epochs=5, batch_size=128)

loss,accuracy = merge_model.evaluate(x_test,y_test, verbose=0)

end=time()

print('Test score:', loss)
print('Test accuracy:', accuracy)
print("training and test time:{:.2f}s".format(end-start))

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=250, kernel_size=2, activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=250, kernel_size=3, activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=250, kernel_size=4, activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=250, kernel_size=5, activation="relu")`
  # Remove the CWD from sys.path while we load stuff.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 300)     31889700    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 199, 250)     150250      embedding_1[1][0]                
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 198, 250)     225250      embedding_1[1][0]                
__________________________________________________________________________________________________
conv1d_4 (

10553/10553 [==============================] - ETA: 6:39 - loss: 0.0053 - acc: 1.000 - ETA: 6:49 - loss: 0.0069 - acc: 1.000 - ETA: 6:44 - loss: 0.0208 - acc: 0.997 - ETA: 6:35 - loss: 0.0177 - acc: 0.998 - ETA: 6:29 - loss: 0.0158 - acc: 0.998 - ETA: 6:27 - loss: 0.0143 - acc: 0.998 - ETA: 6:24 - loss: 0.0191 - acc: 0.997 - ETA: 6:19 - loss: 0.0176 - acc: 0.998 - ETA: 6:14 - loss: 0.0163 - acc: 0.998 - ETA: 6:10 - loss: 0.0155 - acc: 0.998 - ETA: 6:04 - loss: 0.0261 - acc: 0.997 - ETA: 5:59 - loss: 0.0350 - acc: 0.997 - ETA: 5:55 - loss: 0.0327 - acc: 0.997 - ETA: 5:49 - loss: 0.0311 - acc: 0.997 - ETA: 5:43 - loss: 0.0293 - acc: 0.997 - ETA: 5:39 - loss: 0.0279 - acc: 0.998 - ETA: 5:33 - loss: 0.0266 - acc: 0.998 - ETA: 5:29 - loss: 0.0254 - acc: 0.998 - ETA: 5:23 - loss: 0.0245 - acc: 0.998 - ETA: 5:17 - loss: 0.0235 - acc: 0.998 - ETA: 5:12 - loss: 0.0226 - acc: 0.998 - ETA: 5:07 - loss: 0.0267 - acc: 0.998 - ETA: 5:02 - loss: 0.0257 - acc: 0.998 - ETA: 4:57 - loss: 0.0248 - acc: 0

# 敏感性分析

In [5]:
def sen_test(test_slt,MAX_SEQUENCE_LENGTH):
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    x_train, x_test, y_train, y_test = tts(data, labels, test_size=test_slt, random_state=42)
    print ('train docs: '+str(len(x_train)))
    print ('test docs: '+str(len(x_test)))
    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items(): 
        if word in w2v_model:
            embedding_matrix[i] = np.asarray(w2v_model[word],
                                             dtype='float32')
    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True)
    model = Sequential()
    model.add(embedding_layer)
    model.add(Dropout(0.2))
    model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1))
    #model.add(GlobalMaxPooling1D())
    model.add(MaxPooling1D(3))
    model.add(Flatten())
    model.add(Dense(EMBEDDING_DIM, activation='relu'))
    model.add(Dense(labels.shape[1], activation='softmax'))
    model.summary()
    #plot_model(model, to_file='model.png',show_shapes=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.fit(x_train, y_train, validation_split=VALIDATION_SPLIT, epochs=5, batch_size=128)
    loss,accuracy = model.evaluate(x_test,y_test, verbose=0)
    #print('Test score:', loss)
    print('test size is:',test_slt)
    print('max seq length is:',MAX_SEQUENCE_LENGTH)
    print('Test accuracy:', accuracy)
    print("\n"*2)

In [ ]:
test_split_list=[0.1,0.2,0.3]
max_len_list=[100,150,200,250,300]
start=time()
for ts in test_split_list:
    for msl in max_len_list:
        sen_test(ts,msl)
end=time()
print("sensitive test time:{:.2f}s".format(end-start))

train docs: 16962
test docs: 1885
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 300)          31889700  
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 98, 250)           225250    
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 32, 250)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 300)               2400300   
_________________________________________________________________
dense_10 (Dense)             (None, 20)   

13569/13569 [==============================] - ETA: 2:42 - loss: 0.3737 - acc: 0.929 - ETA: 2:39 - loss: 0.3505 - acc: 0.918 - ETA: 2:40 - loss: 0.3090 - acc: 0.927 - ETA: 2:40 - loss: 0.2863 - acc: 0.937 - ETA: 2:38 - loss: 0.2793 - acc: 0.935 - ETA: 2:36 - loss: 0.2666 - acc: 0.942 - ETA: 2:35 - loss: 0.2723 - acc: 0.937 - ETA: 2:34 - loss: 0.2678 - acc: 0.938 - ETA: 2:33 - loss: 0.2621 - acc: 0.941 - ETA: 2:32 - loss: 0.2612 - acc: 0.943 - ETA: 2:31 - loss: 0.2601 - acc: 0.943 - ETA: 2:30 - loss: 0.2599 - acc: 0.946 - ETA: 2:28 - loss: 0.2549 - acc: 0.946 - ETA: 2:26 - loss: 0.2602 - acc: 0.944 - ETA: 2:25 - loss: 0.2574 - acc: 0.944 - ETA: 2:23 - loss: 0.2557 - acc: 0.944 - ETA: 2:21 - loss: 0.2549 - acc: 0.944 - ETA: 2:20 - loss: 0.2631 - acc: 0.944 - ETA: 2:18 - loss: 0.2604 - acc: 0.944 - ETA: 2:17 - loss: 0.2591 - acc: 0.944 - ETA: 2:16 - loss: 0.2604 - acc: 0.942 - ETA: 2:14 - loss: 0.2603 - acc: 0.942 - ETA: 2:13 - loss: 0.2552 - acc: 0.944 - ETA: 2:12 - loss: 0.2546 - acc: 0

13569/13569 [==============================] - ETA: 2:37 - loss: 0.0240 - acc: 1.000 - ETA: 2:44 - loss: 0.0307 - acc: 0.996 - ETA: 2:46 - loss: 0.0344 - acc: 0.992 - ETA: 2:45 - loss: 0.0306 - acc: 0.994 - ETA: 2:45 - loss: 0.0286 - acc: 0.993 - ETA: 2:42 - loss: 0.0284 - acc: 0.993 - ETA: 2:41 - loss: 0.0273 - acc: 0.994 - ETA: 2:39 - loss: 0.0283 - acc: 0.994 - ETA: 2:38 - loss: 0.0262 - acc: 0.994 - ETA: 2:36 - loss: 0.0255 - acc: 0.995 - ETA: 2:35 - loss: 0.0262 - acc: 0.995 - ETA: 2:33 - loss: 0.0263 - acc: 0.995 - ETA: 2:32 - loss: 0.0256 - acc: 0.995 - ETA: 2:30 - loss: 0.0247 - acc: 0.996 - ETA: 2:28 - loss: 0.0237 - acc: 0.996 - ETA: 2:27 - loss: 0.0232 - acc: 0.996 - ETA: 2:25 - loss: 0.0227 - acc: 0.996 - ETA: 2:24 - loss: 0.0228 - acc: 0.997 - ETA: 2:22 - loss: 0.0223 - acc: 0.997 - ETA: 2:20 - loss: 0.0221 - acc: 0.996 - ETA: 2:18 - loss: 0.0219 - acc: 0.997 - ETA: 2:16 - loss: 0.0215 - acc: 0.997 - ETA: 2:15 - loss: 0.0210 - acc: 0.997 - ETA: 2:13 - loss: 0.0206 - acc: 0

13569/13569 [==============================] - ETA: 3:07 - loss: 0.9812 - acc: 0.679 - ETA: 3:05 - loss: 0.8443 - acc: 0.722 - ETA: 3:07 - loss: 0.8448 - acc: 0.731 - ETA: 3:08 - loss: 0.8973 - acc: 0.710 - ETA: 3:07 - loss: 0.8928 - acc: 0.706 - ETA: 3:05 - loss: 0.8576 - acc: 0.722 - ETA: 3:03 - loss: 0.8380 - acc: 0.729 - ETA: 3:01 - loss: 0.8306 - acc: 0.732 - ETA: 2:58 - loss: 0.8316 - acc: 0.734 - ETA: 2:56 - loss: 0.8194 - acc: 0.739 - ETA: 2:55 - loss: 0.8164 - acc: 0.745 - ETA: 2:54 - loss: 0.8089 - acc: 0.748 - ETA: 2:54 - loss: 0.7970 - acc: 0.751 - ETA: 2:52 - loss: 0.7887 - acc: 0.757 - ETA: 2:51 - loss: 0.7772 - acc: 0.758 - ETA: 2:50 - loss: 0.7735 - acc: 0.761 - ETA: 2:48 - loss: 0.7684 - acc: 0.761 - ETA: 2:47 - loss: 0.7627 - acc: 0.764 - ETA: 2:45 - loss: 0.7554 - acc: 0.766 - ETA: 2:42 - loss: 0.7563 - acc: 0.763 - ETA: 2:41 - loss: 0.7503 - acc: 0.764 - ETA: 2:39 - loss: 0.7450 - acc: 0.766 - ETA: 2:37 - loss: 0.7428 - acc: 0.768 - ETA: 2:35 - loss: 0.7486 - acc: 0

13569/13569 [==============================] - ETA: 3:21 - loss: 0.0615 - acc: 0.992 - ETA: 3:15 - loss: 0.0480 - acc: 0.992 - ETA: 3:16 - loss: 0.0485 - acc: 0.989 - ETA: 3:13 - loss: 0.0487 - acc: 0.990 - ETA: 3:11 - loss: 0.0468 - acc: 0.990 - ETA: 3:10 - loss: 0.0467 - acc: 0.990 - ETA: 3:09 - loss: 0.0648 - acc: 0.990 - ETA: 3:06 - loss: 0.0600 - acc: 0.991 - ETA: 3:04 - loss: 0.0570 - acc: 0.991 - ETA: 3:03 - loss: 0.0574 - acc: 0.992 - ETA: 3:00 - loss: 0.0556 - acc: 0.992 - ETA: 2:59 - loss: 0.0552 - acc: 0.992 - ETA: 2:56 - loss: 0.0547 - acc: 0.993 - ETA: 2:55 - loss: 0.0526 - acc: 0.993 - ETA: 2:53 - loss: 0.0522 - acc: 0.993 - ETA: 2:51 - loss: 0.0508 - acc: 0.994 - ETA: 2:49 - loss: 0.0510 - acc: 0.994 - ETA: 2:47 - loss: 0.0510 - acc: 0.993 - ETA: 2:45 - loss: 0.0497 - acc: 0.993 - ETA: 2:43 - loss: 0.0504 - acc: 0.993 - ETA: 2:41 - loss: 0.0519 - acc: 0.992 - ETA: 2:39 - loss: 0.0511 - acc: 0.993 - ETA: 2:38 - loss: 0.0542 - acc: 0.992 - ETA: 2:36 - loss: 0.0542 - acc: 0

...

12061/12061 [==============================] - ETA: 10:09 - loss: 3.0024 - acc: 0.07 - ETA: 6:56 - loss: 3.8631 - acc: 0.0820 - ETA: 5:49 - loss: 3.8529 - acc: 0.078 - ETA: 5:13 - loss: 3.8201 - acc: 0.068 - ETA: 4:51 - loss: 3.7629 - acc: 0.065 - ETA: 4:35 - loss: 3.6803 - acc: 0.063 - ETA: 4:23 - loss: 3.6000 - acc: 0.059 - ETA: 4:13 - loss: 3.5310 - acc: 0.063 - ETA: 4:05 - loss: 3.4638 - acc: 0.067 - ETA: 3:58 - loss: 3.4206 - acc: 0.068 - ETA: 3:53 - loss: 3.3817 - acc: 0.071 - ETA: 3:49 - loss: 3.3495 - acc: 0.071 - ETA: 3:43 - loss: 3.3192 - acc: 0.077 - ETA: 3:39 - loss: 3.2931 - acc: 0.080 - ETA: 3:35 - loss: 3.2705 - acc: 0.085 - ETA: 3:31 - loss: 3.2501 - acc: 0.089 - ETA: 3:27 - loss: 3.2346 - acc: 0.091 - ETA: 3:24 - loss: 3.2190 - acc: 0.092 - ETA: 3:21 - loss: 3.2044 - acc: 0.094 - ETA: 3:18 - loss: 3.1920 - acc: 0.095 - ETA: 3:15 - loss: 3.1816 - acc: 0.093 - ETA: 3:11 - loss: 3.1699 - acc: 0.095 - ETA: 3:08 - loss: 3.1597 - acc: 0.097 - ETA: 3:05 - loss: 3.1501 - acc: 

12061/12061 [==============================] - ETA: 3:50 - loss: 0.0326 - acc: 1.000 - ETA: 3:40 - loss: 0.0237 - acc: 1.000 - ETA: 3:43 - loss: 0.0250 - acc: 1.000 - ETA: 3:42 - loss: 0.0253 - acc: 1.000 - ETA: 3:39 - loss: 0.0300 - acc: 0.996 - ETA: 3:38 - loss: 0.0297 - acc: 0.997 - ETA: 3:35 - loss: 0.0288 - acc: 0.997 - ETA: 3:34 - loss: 0.0273 - acc: 0.998 - ETA: 3:30 - loss: 0.0274 - acc: 0.997 - ETA: 3:26 - loss: 0.0267 - acc: 0.997 - ETA: 3:23 - loss: 0.0258 - acc: 0.997 - ETA: 3:20 - loss: 0.0253 - acc: 0.998 - ETA: 3:17 - loss: 0.0247 - acc: 0.998 - ETA: 3:15 - loss: 0.0250 - acc: 0.998 - ETA: 3:13 - loss: 0.0251 - acc: 0.997 - ETA: 3:11 - loss: 0.0244 - acc: 0.998 - ETA: 3:09 - loss: 0.0247 - acc: 0.998 - ETA: 3:07 - loss: 0.0250 - acc: 0.997 - ETA: 3:05 - loss: 0.0244 - acc: 0.997 - ETA: 3:02 - loss: 0.0243 - acc: 0.998 - ETA: 3:00 - loss: 0.0239 - acc: 0.998 - ETA: 2:57 - loss: 0.0235 - acc: 0.998 - ETA: 2:55 - loss: 0.0278 - acc: 0.998 - ETA: 2:52 - loss: 0.0330 - acc: 0

12061/12061 [==============================] - ETA: 4:13 - loss: 0.2842 - acc: 0.921 - ETA: 4:07 - loss: 0.3101 - acc: 0.918 - ETA: 4:08 - loss: 0.2986 - acc: 0.921 - ETA: 4:09 - loss: 0.2870 - acc: 0.923 - ETA: 4:06 - loss: 0.2896 - acc: 0.923 - ETA: 4:04 - loss: 0.2847 - acc: 0.920 - ETA: 4:02 - loss: 0.2805 - acc: 0.924 - ETA: 4:01 - loss: 0.2789 - acc: 0.927 - ETA: 3:58 - loss: 0.2740 - acc: 0.929 - ETA: 3:54 - loss: 0.2708 - acc: 0.930 - ETA: 3:52 - loss: 0.2746 - acc: 0.929 - ETA: 3:49 - loss: 0.2631 - acc: 0.932 - ETA: 3:46 - loss: 0.2575 - acc: 0.934 - ETA: 3:43 - loss: 0.2573 - acc: 0.934 - ETA: 3:40 - loss: 0.2478 - acc: 0.938 - ETA: 3:37 - loss: 0.2452 - acc: 0.938 - ETA: 3:35 - loss: 0.2396 - acc: 0.939 - ETA: 3:32 - loss: 0.2339 - acc: 0.941 - ETA: 3:30 - loss: 0.2343 - acc: 0.944 - ETA: 3:27 - loss: 0.2305 - acc: 0.944 - ETA: 3:24 - loss: 0.2323 - acc: 0.942 - ETA: 3:22 - loss: 0.2304 - acc: 0.942 - ETA: 3:19 - loss: 0.2336 - acc: 0.942 - ETA: 3:16 - loss: 0.2304 - acc: 0

10553/10553 [==============================] - ETA: 7:04 - loss: 2.9959 - acc: 0.046 - ETA: 4:27 - loss: 3.3756 - acc: 0.046 - ETA: 3:38 - loss: 3.5122 - acc: 0.054 - ETA: 3:12 - loss: 3.4035 - acc: 0.058 - ETA: 2:56 - loss: 3.3540 - acc: 0.059 - ETA: 2:43 - loss: 3.2903 - acc: 0.066 - ETA: 2:34 - loss: 3.2699 - acc: 0.065 - ETA: 2:27 - loss: 3.2392 - acc: 0.062 - ETA: 2:21 - loss: 3.2138 - acc: 0.063 - ETA: 2:16 - loss: 3.1926 - acc: 0.068 - ETA: 2:11 - loss: 3.1713 - acc: 0.070 - ETA: 2:07 - loss: 3.1559 - acc: 0.071 - ETA: 2:04 - loss: 3.1398 - acc: 0.070 - ETA: 2:00 - loss: 3.1267 - acc: 0.072 - ETA: 1:58 - loss: 3.1145 - acc: 0.072 - ETA: 1:55 - loss: 3.1038 - acc: 0.070 - ETA: 1:52 - loss: 3.0930 - acc: 0.070 - ETA: 1:50 - loss: 3.0846 - acc: 0.069 - ETA: 1:47 - loss: 3.0754 - acc: 0.070 - ETA: 1:45 - loss: 3.0645 - acc: 0.072 - ETA: 1:43 - loss: 3.0564 - acc: 0.072 - ETA: 1:41 - loss: 3.0487 - acc: 0.073 - ETA: 1:39 - loss: 3.0406 - acc: 0.074 - ETA: 1:37 - loss: 3.0339 - acc: 0

 9216/10553 [=========================>....] - ETA: 1:57 - loss: 0.0327 - acc: 0.992 - ETA: 1:55 - loss: 0.0412 - acc: 0.996 - ETA: 1:55 - loss: 0.0361 - acc: 0.997 - ETA: 1:55 - loss: 0.0354 - acc: 0.998 - ETA: 1:53 - loss: 0.0350 - acc: 0.996 - ETA: 1:53 - loss: 0.0347 - acc: 0.996 - ETA: 1:51 - loss: 0.0335 - acc: 0.996 - ETA: 1:49 - loss: 0.0331 - acc: 0.997 - ETA: 1:48 - loss: 0.0337 - acc: 0.997 - ETA: 1:46 - loss: 0.0330 - acc: 0.997 - ETA: 1:44 - loss: 0.0337 - acc: 0.997 - ETA: 1:44 - loss: 0.0338 - acc: 0.998 - ETA: 1:43 - loss: 0.0347 - acc: 0.998 - ETA: 1:42 - loss: 0.0338 - acc: 0.998 - ETA: 1:40 - loss: 0.0335 - acc: 0.998 - ETA: 1:39 - loss: 0.0336 - acc: 0.998 - ETA: 1:37 - loss: 0.0347 - acc: 0.997 - ETA: 1:36 - loss: 0.0345 - acc: 0.997 - ETA: 1:34 - loss: 0.0355 - acc: 0.997 - ETA: 1:33 - loss: 0.0351 - acc: 0.997 - ETA: 1:31 - loss: 0.0348 - acc: 0.997 - ETA: 1:30 - loss: 0.0342 - acc: 0.997 - ETA: 1:28 - loss: 0.0351 - acc: 0.997 - ETA: 1:27 - loss: 0.0360 - acc: 0

因为上一次运行到中间出现内存不足(已运行约3小时)，我们从断点(0.3,100)继续：

In [6]:
start=time()
max_len_list=[100,150,200,250,300]
for msl in max_len_list:
    sen_test(0.3,msl)
    
end=time()
print("sensitive test time:{:.2f}s".format(end-start))

train docs: 13192
test docs: 5655
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          31889700  
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 300)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 250)           225250    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 32, 250)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 300)               2400300   
_________________________________________________________________
dense_2 (Dense)              (None, 20)   

10553/10553 [==============================] - ETA: 2:09 - loss: 0.4139 - acc: 0.921 - ETA: 2:10 - loss: 0.4463 - acc: 0.906 - ETA: 2:09 - loss: 0.4238 - acc: 0.914 - ETA: 2:09 - loss: 0.4358 - acc: 0.900 - ETA: 2:08 - loss: 0.4295 - acc: 0.906 - ETA: 2:07 - loss: 0.4032 - acc: 0.916 - ETA: 2:06 - loss: 0.3883 - acc: 0.917 - ETA: 2:06 - loss: 0.3817 - acc: 0.919 - ETA: 2:05 - loss: 0.3797 - acc: 0.920 - ETA: 2:03 - loss: 0.3727 - acc: 0.921 - ETA: 2:01 - loss: 0.3643 - acc: 0.924 - ETA: 2:00 - loss: 0.3572 - acc: 0.926 - ETA: 1:58 - loss: 0.3589 - acc: 0.925 - ETA: 1:56 - loss: 0.3551 - acc: 0.925 - ETA: 1:54 - loss: 0.3526 - acc: 0.924 - ETA: 1:52 - loss: 0.3510 - acc: 0.924 - ETA: 1:50 - loss: 0.3485 - acc: 0.925 - ETA: 1:48 - loss: 0.3457 - acc: 0.924 - ETA: 1:46 - loss: 0.3448 - acc: 0.923 - ETA: 1:45 - loss: 0.3419 - acc: 0.925 - ETA: 1:43 - loss: 0.3457 - acc: 0.923 - ETA: 1:42 - loss: 0.3438 - acc: 0.923 - ETA: 1:40 - loss: 0.3446 - acc: 0.924 - ETA: 1:38 - loss: 0.3400 - acc: 0

10553/10553 [==============================] - ETA: 2:07 - loss: 0.0443 - acc: 0.992 - ETA: 2:06 - loss: 0.0394 - acc: 0.996 - ETA: 2:04 - loss: 0.0329 - acc: 0.997 - ETA: 2:03 - loss: 0.0293 - acc: 0.998 - ETA: 2:01 - loss: 0.0299 - acc: 0.996 - ETA: 1:59 - loss: 0.0307 - acc: 0.997 - ETA: 1:59 - loss: 0.0301 - acc: 0.997 - ETA: 1:57 - loss: 0.0311 - acc: 0.997 - ETA: 1:56 - loss: 0.0325 - acc: 0.996 - ETA: 1:54 - loss: 0.0314 - acc: 0.996 - ETA: 1:52 - loss: 0.0301 - acc: 0.997 - ETA: 1:51 - loss: 0.0303 - acc: 0.997 - ETA: 1:49 - loss: 0.0302 - acc: 0.997 - ETA: 1:47 - loss: 0.0292 - acc: 0.997 - ETA: 1:46 - loss: 0.0304 - acc: 0.996 - ETA: 1:43 - loss: 0.0300 - acc: 0.997 - ETA: 1:42 - loss: 0.0326 - acc: 0.996 - ETA: 1:41 - loss: 0.0314 - acc: 0.997 - ETA: 1:39 - loss: 0.0312 - acc: 0.997 - ETA: 1:38 - loss: 0.0307 - acc: 0.997 - ETA: 1:37 - loss: 0.0305 - acc: 0.997 - ETA: 1:35 - loss: 0.0302 - acc: 0.997 - ETA: 1:34 - loss: 0.0296 - acc: 0.997 - ETA: 1:32 - loss: 0.0291 - acc: 0

10553/10553 [==============================] - ETA: 2:32 - loss: 1.2164 - acc: 0.585 - ETA: 2:30 - loss: 1.3250 - acc: 0.527 - ETA: 2:29 - loss: 1.3045 - acc: 0.526 - ETA: 2:26 - loss: 1.2914 - acc: 0.552 - ETA: 2:25 - loss: 1.2565 - acc: 0.579 - ETA: 2:23 - loss: 1.2374 - acc: 0.589 - ETA: 2:22 - loss: 1.2190 - acc: 0.598 - ETA: 2:19 - loss: 1.1923 - acc: 0.606 - ETA: 2:18 - loss: 1.1689 - acc: 0.613 - ETA: 2:17 - loss: 1.1668 - acc: 0.616 - ETA: 2:16 - loss: 1.1587 - acc: 0.615 - ETA: 2:14 - loss: 1.1653 - acc: 0.612 - ETA: 2:12 - loss: 1.1653 - acc: 0.610 - ETA: 2:10 - loss: 1.1502 - acc: 0.611 - ETA: 2:07 - loss: 1.1440 - acc: 0.615 - ETA: 2:05 - loss: 1.1404 - acc: 0.617 - ETA: 2:03 - loss: 1.1355 - acc: 0.617 - ETA: 2:01 - loss: 1.1278 - acc: 0.620 - ETA: 2:00 - loss: 1.1194 - acc: 0.623 - ETA: 1:58 - loss: 1.1235 - acc: 0.621 - ETA: 1:56 - loss: 1.1193 - acc: 0.625 - ETA: 1:55 - loss: 1.1141 - acc: 0.626 - ETA: 1:53 - loss: 1.1046 - acc: 0.632 - ETA: 1:52 - loss: 1.0975 - acc: 0

10553/10553 [==============================] - ETA: 2:26 - loss: 0.1119 - acc: 0.968 - ETA: 2:25 - loss: 0.1047 - acc: 0.976 - ETA: 2:26 - loss: 0.1003 - acc: 0.981 - ETA: 2:25 - loss: 0.0991 - acc: 0.984 - ETA: 2:24 - loss: 0.1010 - acc: 0.982 - ETA: 2:22 - loss: 0.1023 - acc: 0.981 - ETA: 2:20 - loss: 0.0973 - acc: 0.983 - ETA: 2:19 - loss: 0.0994 - acc: 0.985 - ETA: 2:17 - loss: 0.0998 - acc: 0.986 - ETA: 2:15 - loss: 0.1104 - acc: 0.985 - ETA: 2:14 - loss: 0.1131 - acc: 0.983 - ETA: 2:12 - loss: 0.1113 - acc: 0.984 - ETA: 2:10 - loss: 0.1114 - acc: 0.984 - ETA: 2:08 - loss: 0.1107 - acc: 0.984 - ETA: 2:06 - loss: 0.1099 - acc: 0.984 - ETA: 2:05 - loss: 0.1103 - acc: 0.983 - ETA: 2:03 - loss: 0.1082 - acc: 0.984 - ETA: 2:01 - loss: 0.1050 - acc: 0.985 - ETA: 1:59 - loss: 0.1044 - acc: 0.985 - ETA: 1:58 - loss: 0.1026 - acc: 0.985 - ETA: 1:56 - loss: 0.1013 - acc: 0.986 - ETA: 1:54 - loss: 0.1001 - acc: 0.986 - ETA: 1:52 - loss: 0.0982 - acc: 0.986 - ETA: 1:50 - loss: 0.0974 - acc: 0

...

10553/10553 [==============================] - ETA: 3:20 - loss: 1.5770 - acc: 0.484 - ETA: 3:19 - loss: 1.5198 - acc: 0.519 - ETA: 3:18 - loss: 1.4919 - acc: 0.528 - ETA: 3:18 - loss: 1.4688 - acc: 0.541 - ETA: 3:16 - loss: 1.4562 - acc: 0.545 - ETA: 3:14 - loss: 1.4577 - acc: 0.540 - ETA: 3:12 - loss: 1.4303 - acc: 0.548 - ETA: 3:11 - loss: 1.4229 - acc: 0.546 - ETA: 3:08 - loss: 1.4086 - acc: 0.549 - ETA: 3:04 - loss: 1.4164 - acc: 0.546 - ETA: 3:02 - loss: 1.4093 - acc: 0.551 - ETA: 2:58 - loss: 1.4036 - acc: 0.551 - ETA: 2:56 - loss: 1.4071 - acc: 0.545 - ETA: 2:53 - loss: 1.3936 - acc: 0.551 - ETA: 2:50 - loss: 1.3775 - acc: 0.555 - ETA: 2:47 - loss: 1.3666 - acc: 0.558 - ETA: 2:45 - loss: 1.3455 - acc: 0.565 - ETA: 2:43 - loss: 1.3383 - acc: 0.569 - ETA: 2:40 - loss: 1.3251 - acc: 0.574 - ETA: 2:37 - loss: 1.3117 - acc: 0.577 - ETA: 2:35 - loss: 1.3045 - acc: 0.579 - ETA: 2:32 - loss: 1.2978 - acc: 0.581 - ETA: 2:30 - loss: 1.2899 - acc: 0.583 - ETA: 2:27 - loss: 1.2827 - acc: 0

10553/10553 [==============================] - ETA: 3:14 - loss: 0.0818 - acc: 0.984 - ETA: 3:14 - loss: 0.1006 - acc: 0.984 - ETA: 3:08 - loss: 0.1118 - acc: 0.976 - ETA: 3:06 - loss: 0.1062 - acc: 0.982 - ETA: 3:04 - loss: 0.1001 - acc: 0.982 - ETA: 3:03 - loss: 0.0980 - acc: 0.983 - ETA: 3:00 - loss: 0.0995 - acc: 0.983 - ETA: 2:59 - loss: 0.0996 - acc: 0.983 - ETA: 2:57 - loss: 0.0984 - acc: 0.984 - ETA: 2:55 - loss: 0.0979 - acc: 0.985 - ETA: 2:53 - loss: 0.0963 - acc: 0.986 - ETA: 2:51 - loss: 0.0957 - acc: 0.987 - ETA: 2:48 - loss: 0.0970 - acc: 0.986 - ETA: 2:46 - loss: 0.0944 - acc: 0.987 - ETA: 2:44 - loss: 0.0914 - acc: 0.988 - ETA: 2:41 - loss: 0.0903 - acc: 0.988 - ETA: 2:39 - loss: 0.0894 - acc: 0.988 - ETA: 2:37 - loss: 0.0880 - acc: 0.987 - ETA: 2:34 - loss: 0.0880 - acc: 0.987 - ETA: 2:31 - loss: 0.0882 - acc: 0.986 - ETA: 2:29 - loss: 0.0868 - acc: 0.986 - ETA: 2:27 - loss: 0.0861 - acc: 0.986 - ETA: 2:24 - loss: 0.0858 - acc: 0.987 - ETA: 2:22 - loss: 0.0858 - acc: 0

10553/10553 [==============================] - ETA: 9:09 - loss: 3.0042 - acc: 0.070 - ETA: 6:29 - loss: 3.6090 - acc: 0.058 - ETA: 5:30 - loss: 3.7970 - acc: 0.049 - ETA: 4:59 - loss: 3.7390 - acc: 0.046 - ETA: 4:39 - loss: 3.6374 - acc: 0.056 - ETA: 4:25 - loss: 3.5963 - acc: 0.058 - ETA: 4:15 - loss: 3.5274 - acc: 0.058 - ETA: 4:07 - loss: 3.4794 - acc: 0.059 - ETA: 4:01 - loss: 3.4350 - acc: 0.059 - ETA: 3:55 - loss: 3.3947 - acc: 0.060 - ETA: 3:49 - loss: 3.3571 - acc: 0.061 - ETA: 3:44 - loss: 3.3294 - acc: 0.061 - ETA: 3:40 - loss: 3.3007 - acc: 0.063 - ETA: 3:35 - loss: 3.2766 - acc: 0.064 - ETA: 3:31 - loss: 3.2573 - acc: 0.065 - ETA: 3:27 - loss: 3.2400 - acc: 0.066 - ETA: 3:23 - loss: 3.2236 - acc: 0.067 - ETA: 3:20 - loss: 3.2092 - acc: 0.066 - ETA: 3:16 - loss: 3.1971 - acc: 0.065 - ETA: 3:12 - loss: 3.1851 - acc: 0.068 - ETA: 3:07 - loss: 3.1748 - acc: 0.067 - ETA: 3:04 - loss: 3.1646 - acc: 0.068 - ETA: 3:00 - loss: 3.1539 - acc: 0.069 - ETA: 2:56 - loss: 3.1463 - acc: 0

10553/10553 [==============================] - ETA: 3:41 - loss: 0.3784 - acc: 0.890 - ETA: 3:42 - loss: 0.3827 - acc: 0.894 - ETA: 3:37 - loss: 0.3957 - acc: 0.888 - ETA: 3:34 - loss: 0.3955 - acc: 0.890 - ETA: 3:31 - loss: 0.3697 - acc: 0.898 - ETA: 3:29 - loss: 0.3674 - acc: 0.898 - ETA: 3:24 - loss: 0.3672 - acc: 0.896 - ETA: 3:22 - loss: 0.3556 - acc: 0.906 - ETA: 3:19 - loss: 0.3569 - acc: 0.908 - ETA: 3:15 - loss: 0.3543 - acc: 0.910 - ETA: 3:12 - loss: 0.3406 - acc: 0.915 - ETA: 3:09 - loss: 0.3350 - acc: 0.918 - ETA: 3:07 - loss: 0.3281 - acc: 0.921 - ETA: 3:04 - loss: 0.3279 - acc: 0.920 - ETA: 3:02 - loss: 0.3289 - acc: 0.918 - ETA: 2:59 - loss: 0.3233 - acc: 0.921 - ETA: 2:57 - loss: 0.3197 - acc: 0.922 - ETA: 2:55 - loss: 0.3147 - acc: 0.923 - ETA: 2:52 - loss: 0.3187 - acc: 0.921 - ETA: 2:49 - loss: 0.3122 - acc: 0.924 - ETA: 2:46 - loss: 0.3106 - acc: 0.924 - ETA: 2:44 - loss: 0.3075 - acc: 0.924 - ETA: 2:41 - loss: 0.3085 - acc: 0.923 - ETA: 2:38 - loss: 0.3114 - acc: 0

10553/10553 [==============================] - ETA: 3:55 - loss: 0.0228 - acc: 1.000 - ETA: 3:52 - loss: 0.0226 - acc: 1.000 - ETA: 3:47 - loss: 0.0206 - acc: 1.000 - ETA: 3:40 - loss: 0.0262 - acc: 0.998 - ETA: 3:35 - loss: 0.0287 - acc: 0.998 - ETA: 3:32 - loss: 0.0261 - acc: 0.998 - ETA: 3:29 - loss: 0.0267 - acc: 0.998 - ETA: 3:26 - loss: 0.0279 - acc: 0.999 - ETA: 3:22 - loss: 0.0293 - acc: 0.998 - ETA: 3:19 - loss: 0.0279 - acc: 0.998 - ETA: 3:17 - loss: 0.0277 - acc: 0.998 - ETA: 3:15 - loss: 0.0270 - acc: 0.998 - ETA: 3:12 - loss: 0.0347 - acc: 0.998 - ETA: 3:10 - loss: 0.0354 - acc: 0.997 - ETA: 3:07 - loss: 0.0354 - acc: 0.997 - ETA: 3:05 - loss: 0.0350 - acc: 0.998 - ETA: 3:02 - loss: 0.0350 - acc: 0.997 - ETA: 2:59 - loss: 0.0347 - acc: 0.997 - ETA: 2:56 - loss: 0.0338 - acc: 0.997 - ETA: 2:53 - loss: 0.0326 - acc: 0.997 - ETA: 2:50 - loss: 0.0324 - acc: 0.997 - ETA: 2:48 - loss: 0.0321 - acc: 0.997 - ETA: 2:45 - loss: 0.0315 - acc: 0.997 - ETA: 2:42 - loss: 0.0306 - acc: 0